# 1. Prerequisites

## Software

- BUSCO (https://busco.ezlab.org/) (compleasm(https://github.com/huangnengCSU/compleasm), compleasm: a faster and more accurate reimplementation of BUSCO)
- RepeatMasker, RepeatModeler (http://www.repeatmasker.org/)
- HISAT2 (http://daehwankimlab.github.io/hisat2/)
- StringTie (http://ccb.jhu.edu/software/stringtie/)
- TransDecoder (https://github.com/TransDecoder/TransDecoder)
- BRAKER (https://github.com/Gaius-Augustus/BRAKER)
- AUGUSTUS (https://github.com/Gaius-Augustus/Augustus)
- GeneMark (http://topaz.gatech.edu/GeneMark/)
- BAMTOOLS (https://github.com/pezmaster31/bamtools)
- SAMTOOLS (http://www.htslib.org/)
- ProtHint (https://github.com/gatech-genemark/ProtHint)
- DIAMOND (http://github.com/bbuchfink/diamond/)
- NCBI BLAST+ (https://blast.ncbi.nlm.nih.gov/Blast.cgi)
- miniprot (https://github.com/lh3/miniprot)
- EVidenceModeler (https://github.com/EVidenceModeler/EVidenceModeler/wiki)
- PASA (https://github.com/PASApipeline/PASApipeline/wiki)
- Blat (http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/blat/blat)
- fasta (http://faculty.virginia.edu/wrpearson/fasta/fasta36/fasta-36.3.8g.tar.gz)
- gffread (https://github.com/gpertea/gffread)

## DataSet

- RNA-seq (https://www.ncbi.nlm.nih.gov/sra/)
- Homology protein (https://bioinf.uni-greifswald.de/bioinf/partitioned_odb11/)


# 2. Genome assessment

## BUSCO v5

Input:

- genome.fa
- Linage dataset `insecta_odb10`


In [ ]:
busco --cpu 28 \
	-l /gpfs/home/meiyang/opt/insecta_odb10 \
	-m genome --force -o busco \
	-i genome.fa \
	--offline

In [ ]:
cat out/short_summary.specific.insecta_odb10.out.txt

# 3. Repeat annotation and genome mask


## 3.1 RepeatModeler v2 & RepeatMasker

Input:

- genome.fa

Output:

- genome.fa.masked

Building reference repeat database

In [ ]:
# RepeatMasker
famdb.py -i Libraries/RepeatMaskerLib.h5 families -f embl  -a -d Insecta  > Insecta_ad.embl
util/buildRMLibFromEMBL.pl Insecta_ad.embl > Insecta_ad.fa

# RepeatModeler
mkdir 01_RepeatModeler
BuildDatabase -name GDB -engine ncbi ../genome.fa > BuildDatabase.log
RepeatModeler -engine ncbi -pa 28 -database GDB -LTRStruct > RepeatModele.log
cd ../

Running RepeatMasker for genome masking

In [ ]:
mkdir 02_RepeatMasker
cat 01_RepeatModeler/GDB-families.fa Insecta_ad.fa > repeat_db.fa

# RepeatMasker (output: genome.fa.masked)
RepeatMasker -xsmall -gff -html -lib repeat_db.fa -pa 28 genome.fa > RepeatMasker.log

## 3.2 EDTA

In [ ]:
EDTA.pl --genome female.fa --species others --sensitive 1 --anno 1 --evaluate 1 --threads 30

# 4. Gene prediction

## 4.1 RNA-seq based gene prediction

### 4.1.1 HISAT2 & StringTie

Input:

- masked geome (genome.fa.masked)
- transcriptome

In [ ]:
# Build genome index
hisat2-build -p 28 genome.fa genome

In [ ]:
# Mapping to genome
# single end
# hisat2 -p 28 -x genome --dta -U reads.fq | samtools sort -@ 28 > reads.bam 

## OR ##

# paired end
hisat2 -p 28 -x genome --dta -1 reads_1.fq -2 reads_2.fq | samtools sort -@ 28 > reads.bam

In [ ]:
# GTF merging
samtools merge -@ 28 merged.bam `ls *bam`
stringtie -p 28 -o stringtie.gtf merged.bam

## OUTPUT: stringtie.gtf

### 4.1.2 TransDecoder

[TransDecoder Wiki](https://github.com/TransDecoder/TransDecoder/wiki)

Input:

- masked geome (genome.fa.masked)
- stringtie.gtf

Predicting coding regions from a transcript fasta file
The 'TransDecoder' utility is run on a fasta file containing the target transcript sequences. The simplest usage is as follows:

In [ ]:
# Construct the transcript fasta file using the genome and the transcripts.gtf
util/gtf_genome_to_cdna_fasta.pl stringtie.gtf genome.fa > transcripts.fasta
# Convert the transcript structure GTF file to an alignment-GFF3 formatted file (for compatibility)
util/gtf_to_alignment_gff3.pl stringtie.gtf > transcripts.gff3

# Step 1: extract the long open reading frames
TransDecoder.LongOrfs -t transcripts.fasta

# Step 2: (optional) homology search.
# Optionally, identify ORFs with homology to known proteins via blast or pfam searches.
blastp -query transdecoder_dir/longest_orfs.pep -db uniprot_sprot.fasta  -max_target_seqs 1 -outfmt 6 -evalue 1e-5 -num_threads 28 > blastp.outfmt6
hmmscan --cpu 28 --domtblout pfam.domtblout Pfam-A.hmm transdecoder_dir/longest_orfs.pep

# Step 3: predict the likely coding regions.
TransDecoder.Predict -t transcripts.fasta --retain_pfam_hits pfam.domtblout --retain_blastp_hits blastp.outfmt6
# Finally, generate a genome-based coding region annotation file.
util/cdna_alignment_orf_to_genome_orf.pl transcripts.fasta.transdecoder.gff3 transcripts.gff3 transcripts.fasta > transcripts.fasta.transdecoder.genome.gff3

mv transcripts.fasta.transdecoder.genome.gff3 transcript_alignments.gff3

## OUTPUT: transcript_alignments.gff3

## 4.2 Ab initio gene prediction

### BRAKER v3

Input:

- masked geome (genome.fa.masked)
- homology protein (OrthoDB), Arthropoda.fa

Parameters

``` bash
--species=<species_name>
--min_contig, less than genome N50
```

In [ ]:
braker.pl --genome=genome.fa \
	--species=Sfru \
	--prot_seq=Arthropoda.fa \
	--bam=merged.bam \
	--threads 30 \
	--gff3 --workingdir=out

python gff_rename.py braker.gff3 sfru > gene_predictions.gff3

## 4.3 Homology-based gene prediction

### miniprot

Input:

- masked geome (genome.fa.masked)
- homology protein (OrthoDB), Arthropoda.fa

In [ ]:
miniprot -t28 -d genome.mpi genome.fa.masked 
miniprot -It28 --gff genome.mpi protein.fasta > miniprot.gff3

## OUTPUT: miniprot.gff3

# 5. EVidenceModeler (EVM)

Input:

- masked geome (genome.fa.masked)
- weights.txt
- GFF3 file
    - gene_predictions.gff3 (from step 4.2)
    - protein_alignments.gff3 (from EVidenceModeler)
    - transcript_alignments.gff3 (from step 4.1.2)

Example of weights.txt

```
PROTEIN	miniprot	5
ABINITIO_PREDICTION	BRAKER3	10
OTHER_PREDICTION	transdecoder	10
```

In [ ]:
# The gff3 file of miniprot should be reformated.
python ~/software/EVidenceModeler-v2.1.0/EvmUtils/misc/miniprot_GFF_2_EVM_alignment_GFF3.py miniprot.gff3 > protein_alignmentss.gff3

In [ ]:
EVidenceModeler \
	--sample_id speceis \
	--genome genome.fa \
	--weights weights.txt  \
	--gene_predictions gff/gene_predictions.gff3 \
	--protein_alignments gff/protein_alignments.gff3 \
	--transcript_alignments gff/transcript_alignments.gff3 \
	--segmentSize 100000 --overlapSize 10000 --CPU 20


# 6. OGS annotation

## 6.1 OGS annotation updates

### 6.1.1 PASApipeline

- masked geome (genome.fa)
- species.evm.gff3
- stringtie.gtf

In [ ]:
# PASA alignment Assembly
util/gtf_genome_to_cdna_fasta.pl stringtie.gtf genome.fa > transcripts.fasta
bin/seqclean transcripts.fasta

Transcripts alignments, alignAssembly.config, set up the mysql database name; CPU <= 16

In [ ]:
Launch_PASA_pipeline.pl -c alignAssembly.config -C -R -g genome.fa -t transcripts.fasta.clean -T -u transcripts.fasta --ALIGNERS blat --CPU 16

# two cycles !!! of annotation loading, annotation comparison, and annotation updates
# check gff3
misc_utilities/pasa_gff3_validator.pl species.evm.gff3

# load annotation
scripts/Load_Current_Gene_Annotations.dbi -c alignAssembly.config -g genome.fa -P species.evm.gff3

# update
# annotCompare.config, set up the mysql database name same as alignAssembly.config
Launch_PASA_pipeline.pl -c annotCompare.config -A -g genome.fa -t transcripts.fasta.clean

### 6.1.2 peaks2utr

In [ ]:
peaks2utr -p 20 species.evm.gff3 merged.bam

## 6.2 Collect GFF, cds, PEP

Input:

- gene_structures_post_PASA_updates.gff3

Output:

- Sfru.gff3 (Sfru_no_alt.gff3)
- cds.fa (cds_no_alt.fa)
- pep.fa (pep_no_alt.fa)

In [ ]:
# rename gff3
# species name, Sfru
python gff_rename.py gene_structures_post_PASA_updates.gff3 Sfru > Sfru.gff3

# collect
gffread Sfru.gff3 -g genome.fa -x Sfru_cds.fa -y Sfru_pep.fa

# collect no alt gff, cds, pep 
python Collect_no_alt.py pep.fa cds.fa Sfru.gff3
# no_alt.gff3, cds_no_alt.fa, pep_no_alt.fa

## 6.3 Function annotation

eggNOG-mapper

pep_no_alt.fa

http://eggnog-mapper.embl.de/

PANNZER
http://ekhidna2.biocenter.helsinki.fi/sanspanz/